In [1]:
import platform

from selenium import webdriver
from selenium.webdriver.firefox.service import Service as FirefoxService
from selenium.webdriver.firefox.options import Options as FFOptions

from selenium.webdriver.edge.service import Service as EdgeService
from selenium.webdriver.edge.options import Options as EDOptions

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import pandas as pd
from IPython.display import display

from datetime import datetime, timedelta


# ===================================================

def combinacao_dia_semana(dia_semana,verbose=False):

    dia_semana = dia_semana
    dia_semana = dia_semana.replace(' ','') # Retira os espaços

    if len(dia_semana.split('-')) > 1:
        temp1 = [
            dia_semana.split('-')[0].capitalize() +"-"+dia_semana.split('-')[1].lower(),      # Segunda-feira
            dia_semana.split('-')[0].capitalize() +"-"+dia_semana.split('-')[1].capitalize(), # Segunda-Feira
            dia_semana.split('-')[0].capitalize() +" "+dia_semana.split('-')[1].lower(),      # Segunda feira
            dia_semana.split('-')[0].capitalize() +" "+dia_semana.split('-')[1].capitalize(), # Segunda Feira
        ]
    else:
        temp1 = []

    a_list = [
        dia_semana.lower(),                    # segunda-feira
        dia_semana.upper(),                    # SEGUNDA-FEIRA
        dia_semana.replace("-"," "),           # Segunda Feira
        dia_semana.lower().replace("-"," "),   # segunda feira
        dia_semana.upper().replace("-"," "),   # SEGUNDA FEIRA
        dia_semana.split('-')[0],              # Segunda
        dia_semana.lower().split('-')[0],      # segunda
        dia_semana.upper().split('-')[0]]      # SEGUNDA

    if 'ç' in dia_semana:
        temp = [i.replace("ç",'c').replace("Ç","C") for i in a_list]
    elif 'á' in dia_semana.lower():
        temp = [i.replace("á",'a').replace("À","A") for i in a_list]
    else:
        temp = []

    a_list = a_list + temp1 + temp

    a_list = list(set(a_list)) # O set(...) retira os elementos repetidos 
    return a_list

def dia_semana_to_weekday(dia):
    """Converte um dia da semana em pt para formato do
    datetime.weekday sendo que a segunda=0

    Args:
        dia (stt): Dia da semana por extenso
    """
    if dia in combinacao_dia_semana('Segunda-Feira'):
        return 0
    if dia in combinacao_dia_semana('Terça-Feira'):
        return 1
    if dia in combinacao_dia_semana('Quarta-Feira'):
        return 2
    if dia in combinacao_dia_semana('Quinta-Feira'):
        return 3
    if dia in combinacao_dia_semana('Sexta-Feira'):
        return 4
    if dia in combinacao_dia_semana('Sábado-Feira'):
        return 5
    if dia in combinacao_dia_semana('Domingo-Feira'):
        return 6

def gera_datas(dia_inicial="01/08/2024", dia_de_reuniao=None):
    """Gera todas as datas de reunião do grupo entre uma 
    data de inicio e a data atual. Primeiro pega a data de início ('dia_inicial')
    e identifica por meio do 'dia_de_reuniao' qual o dia da semana que o grupo se 
    reune e procura qual o primeiro desses dias após a data de início.

    Args:
        data_inicio (str): No formato "dia/mes/ano". Defaults to "01/08/2024".
        dia_de_reuniao (str ou int): um dia da semana por extenso. Ex: 'Quarta-Feira'
    returns: 
        list de strings: contendo as datas no formato dia/mes/ano
    """    

    # Parse da data de inicio para datetime
    data_inicio_sting = dia_inicial
    data_inicio= datetime.strptime(data_inicio_sting,"%d/%m/%Y")

    # Parse da data de fim para datetime
    data_fim = datetime.today()

    # Seta o timedelta para 1 semana
    dt = timedelta(weeks=1)

    # Procura a primeira data de reunião (ex: quarta)
    # Após a data de início

    weekday = dia_semana_to_weekday(dia_de_reuniao) # Helper Function
    dia_0 = data_inicio
    while(dia_0.weekday() != weekday):
        dia_0 += timedelta(days=1)

    # Gera a lista de datas
    datas = []
    dia=dia_0
    while(dia <= data_fim):
        datas.append(dia.date().strftime("%d/%m/%Y"))
        dia += dt

    return datas

##### ----- Classe WOP ------ ################################
##
class WOP:
    """Cria um objeto WOP. 
    1) [automático] Ler a lista de grupos 
    1) [automático] Iniciar o Selenium com WOP.inicia_selenium()
    2) [automático] Fazer o login com WOP.login(usuario,senha)

    No momento somente firefox no linux e edge no windows estão implementados
    """
    def __init__(self, usuario, senha, path_grupos_csv = './grupos.csv', path_driver=None):
        self.path_grupos_csv = path_grupos_csv
        self.path_driver = path_driver
        self.usuario = usuario
        self.senha = senha
        self.grupos_df = self.get_grupos_df()
        self.dias_reuniao = {
                            "São Brás": {
                                "Jovens":"Sábado",
                                "Casados":"Quarta-Feira",
                                "Adultos":"Quinta-Feira"
                                },
                            
                            "Ananindeua": {
                                "Jovens":"Sábado",
                                "Casados":"Quarta-Feira",
                                "Adultos":"Quinta-Feira"
                                }
                            
                            }

        self.driver = self.inicializa_selenium()
        self.faz_login()

    def inicializa_selenium(self):
        """Inicializa o selenium. No momento somente firefox no linux e edge no windows são suportados
        return: selenium.webdriver object
        """

        if self.path_driver: # Se for passado o path do driver
            
            # Usa esse path
            driver_path = self.path_driver 
            
            # E decide qual sistema usar
            if "geckodriver" in driver_path.lower():
                sistema = "Linux"
            elif "msedgedriver" in driver_path.lower():
                sistema = "Windows"

        else: # Se não for passado o driver

            sistema = platform.system() # Identifica o sistema
            driver_path = None # Garante que o driver seja None para que o código decida

        # De posse do sistema
        if sistema == "Windows":

            driver_path = "./msedgedriver.exe" if not driver_path else driver_path
            
            options = EDOptions()
            #options.add_argument('--headless')  # Se quiser rodar o Firefox em modo headless
            service = EdgeService(executable_path=driver_path)
            driver = webdriver.Edge(service=service, options=options)
            
        elif sistema == "Linux":

            driver_path = "geckodriver" if not driver_path else driver_path
            
            options = FFOptions()
            #options.add_argument('--headless')  # Se quiser rodar o Firefox em modo headless
            service = FirefoxService(executable_path=driver_path)
            driver = webdriver.Firefox(service=service, options=options)

        elif sistema == "Darwin":
            raise Exception("MacOS ainda não implementado.")
            
        else:
            print("Sistema operacional não reconhecido.")
        

        print(f"| Selenium Iniciado no {sistema} com {driver_path}.")
        return driver

    def faz_login(self):
        print("| Fazendo Login")

        driver = self.driver
        pagina_de_login='https://wop.obra.shalom.tec.br/login'
        driver.get(pagina_de_login)

        username_field = driver.find_element(By.NAME, 'username')  # Use o nome correto do campo de usuário
        password_field = driver.find_element(By.NAME, 'password')  # Use o nome correto do campo de senha

        username_field.clear()
        password_field.clear()
        username_field.send_keys(usuario)
        password_field.send_keys(senha)

        login_button = driver.find_element(By.XPATH, '//button[@type="submit"]')  # Use o seletor correto para o botão de login
        login_button.click()

        print("|-- Login Ok")
        display(self.grupos_df)
        #return 1

    def get_grupos_df(self):
        """Carrega a lista e informações do grupos a partir de um csv

        Returns:
            pd.Dataframe: Dataframe com os grupos
        """        
        grupos_df = pd.read_csv(self.path_grupos_csv).sort_values(by=["CEV", "publico", "nome"], ignore_index=True)
        return grupos_df

    # Método para pegar as ovelhas do grupo
    def get_ovelhas(self, grupo_id, to_df=True, exporta=False, exporta_path = './grupos'):
        """Entra no link de um determinado grupo e cria um list de list contendo 
    as ovelhasjá cadastradas nesse grupo de oração.
    Args:
        to_df: booleano que indica se o valor deve ser retornado já em um pandas df
    Returns:
        um list de lists no formato [[...], [...], [...]] a menos que
        seja indicado to_df
    """ 
        id_do_grupo = grupo_id
        grupos_df = self.grupos_df

        grupo_selecionado = grupos_df.loc[grupos_df["id"]==grupo_id]

        if not grupo_selecionado.empty:
            nome_grupo = grupo_selecionado.iloc[0]['nome']
            CEV_grupo = grupo_selecionado.iloc[0]['CEV']
        else:
            nome_grupo = None
            print(f"Grupo com id {id_do_grupo} não encontrado.")
            raise "Grupo Não encontrado"
        
        exporta_str = f"{exporta_path}/{CEV_grupo}_{id_do_grupo}_{nome_grupo}.csv"

        print(f"| Obtendo a lista de membros do grupo {nome_grupo} de {CEV_grupo}")

        driver = self.driver
        driver.get(f"https://wop.obra.shalom.tec.br/secured/local/grupodeoracao/{ id_do_grupo }/gerenciar")
        

        try:
            # Espere até que o elemento esteja visível (aguarde até 10 segundos)
            xpath= "/html/body/app-root/grupooracao-local-manage/secured/body/div/span/div/section[2]/div/div[4]/div/div[3]/div/div/table-component/div/div[2]/table/tbody"
            tabela = WebDriverWait(driver, 30).until(
                EC.visibility_of_element_located((By.XPATH, xpath))
            )
        except:
            print("|!- O elemento não foi carregado a tempo")
            df = pd.DataFrame( columns=["user id","Nome", "Data de ingresso", "Status"])
            df.to_csv(exporta_str, index=False)
            return df

        linhas = tabela.find_elements(By.TAG_NAME, 'tr')

        # Cria um list com as ovelhas encontradas na tabela
        membros_list =[]
        for linha in linhas:
            colunas = linha.find_elements(By.TAG_NAME, 'td')
            dados = [coluna.text for coluna in colunas if coluna.text != '']
            membros_list.append(dados)

        membros_df = pd.DataFrame(membros_list, columns=["user id","Nome", "Data de ingresso", "Status"])
        membros_df = membros_df.sort_values(by='Nome')

        if exporta:
            membros_df.to_csv(exporta_str, index=False)
            print("|-- Exportado para"+exporta_str)
            

        # Decide qual será a saída
        if to_df:
            return membros_df
        else:
            return membros_list

    def get_frequencia(self, id):
        """Obtem a frequência do grupo dado pelo id

        Args:
            id (_type_): _description_

        Returns:
            _type_: _description_
        """        
        id_do_grupo = id
        
        driver = self.driver

        driver.get(f" https://wop.obra.shalom.tec.br/secured/local/grupodeoracao/{ id_do_grupo }/presenca")

        # Pega as frequencias
        try:
            # Espere até que o elemento esteja visível (aguarde até 10 segundos)
            #xpath= "/html/body/app-root/presenca-grupooracao-local/secured/body/div/span/div/section[2]/div/div/div/div/div[2]/div/div[2]/table"
            css_selector = ".table > tbody:nth-child(2)"
            tabela = WebDriverWait(driver, 5).until(
                EC.visibility_of_element_located((By.CSS_SELECTOR,css_selector))#((By.XPATH, xpath))
            )
        except:
            print("|-------- Frequência não encontrada")
            return []
        
        frequencias = tabela.text.splitlines()
        return frequencias

    def get_dias_reuniao(self,CEV,publico):
        return self.dias_reuniao[CEV][publico]

    def gera_mapa_frequencias(self, CEV, publico, data_inicial="01/08/2024", exporta=False, exporta_path = './freq'):
        """Cria o mapa de lançamento de frequência por público. Utilizando o parametro 'publico',
        o sistema cria todas as datas que correspondem ao dia da semana de cada público e gera a 
        partir de 'data_inicial'. Por exemplo, para os Adultos que tem reunião na Quinta-Feira, 
        o método identifica a primeira quinta-feira a patir de 'data_inicial' e gera todas as outras
        quintas-feira subsequentes até a data de hoje. Após isso vai no WOP e pega a lista de frequencia
        de cada grupo de oração do determinado CEV e organiza num pandas DataFrame.

        Args:
            publico (str): um de ["Adultos", "Casados", "Jovens"]
            data_inicial (str, optional): Defaults to "01/08/2024".
            CEV (str, optional): Default to 'São Brás'

        Returns:
            Pandas DataFrame: Mapa de lançamento de frequências do público e CEV em questão.
        """
        print(f"| Gerando o mapa de frequências do grupo de {publico}")

        # Pega a lista de grupos de um determinado publico
        grupos_df = self.grupos_df

        # Filtra somente o publico e CEV em questão
        grupos_df = grupos_df.loc[(grupos_df['publico']==f'{publico}') & (grupos_df["CEV"]==CEV)]

        # Obtem o nome de todos os grupos encontrados para entrar nas colunas do dataframe
        # A primeira coluna é a data e as demais são os grupos.
        grupos_nomes = ["Data"]+grupos_df.nome.tolist()

        # Gera as datas de reunião entre a data inicial e final
        datas = gera_datas(data_inicial, dia_de_reuniao=self.get_dias_reuniao(CEV,publico))

        # Inicializa o dataframe com a seguinte estrutura
        # index | Datas | Grupo 1 | Grupo 2 | etc. |
        df = pd.DataFrame(columns= [grupos_nomes])

        # Preenche as datas
        df["Data"] = datas
        df.fillna("",inplace=True)

        # Agora percorre cada coluna (grupo) e verifica se a data da linha está contida
        # nas frequências obtidas no WOP
        for i,grupo in enumerate(grupos_nomes[1:]):
            grupo_nome = grupo
            N = len(grupos_nomes[1:])
            print(f"|-- [{i+1}/{N}] Obtendo frequência do grupo {grupo_nome}")

            # A partir do nome, pega o id
            grupo_id = grupos_df.loc[(grupos_df["nome"]==grupo_nome)]
            grupo_id = grupo_id.iloc[0]["id"]
            
            # passa para o método de obter as frequências lançadas
            frequencias_do_grupo = self.get_frequencia(grupo_id)
            if frequencias_do_grupo:
                df[grupo_nome] = df.apply(lambda x: "Lançada" if x.Data in frequencias_do_grupo else "Não Lançada", axis=1)
            else:
                df[grupo_nome] = "Não Lançada"

        if exporta:
            CEV_filename = CEV.lower().replace("ã","a").replace("á","a").replace(" ","")
            df.to_csv(f"{exporta_path}/{CEV_filename}_{publico}_mapa_frequencias.csv", index=False)
            print(f"|-------- Exportado para {exporta_path}/{CEV_filename}_{publico}_mapa_frequencias.csv")
        return df


    def id_to_nome(self,grupo_id):
        """Helper Function que pega o nome a partir do id do grupo
        Args:
            grupo_id (_type_): _description_
        Returns:
            _type_: _description_
        """    
        # Inicializa o nome do grupo com base no id
        id_do_grupo = grupo_id
        grupo_selecionado = self.grupos_df[self.grupos_df['id'] == id_do_grupo]
        if not grupo_selecionado.empty:
            nome = grupo_selecionado.iloc[0]['nome']
        else:
            nome = None
            print(f"Grupo com id {self.id} não encontrado.")
        return nome

In [2]:
# Pega o usuaridia_semana_to_weekday('Segunda')o e senha 
with open('arquivo','r') as arquivo:
    conteudo = arquivo.readlines()
usuario, senha = [conteudo[0].strip(),conteudo[1].strip()] 

In [4]:
WOP = WOP(usuario, senha)

| Selenium Iniciado no Windows com ./msedgedriver.exe.
| Fazendo Login
|-- Login Ok


,id,nome,fase,publico,CEV
0,964,Fiat,Kerygma,Adultos,Ananindeua
1,326,Kadosh,Metanoia,Adultos,Ananindeua
2,725,Pantocrator,Kerygma,Adultos,Ananindeua
3,325,São Tomé,Kerygma,Adultos,Ananindeua
4,324,Israel,fase,Jovens,Ananindeua
5,327,Mãe da Misericórdia,Kerygma,Jovens,Ananindeua
6,727,Santo Agostinho,Kerygma,Jovens,Ananindeua
7,1148,Bom Pastor,Kerygma,Adultos,São Brás
8,301,Fiat,Kerygma,Adultos,São Brás
9,484,Filhas da Imaculada,Koinonia,Adultos,São Brás


In [ ]:
for i,publico in enumerate(["Adultos", "Jovens", "Casados"]):
    print(f"[{i+1}/{3}]")
    WOP.gera_mapa_frequencias(CEV="São Brás", publico=publico, exporta=True)

In [ ]:
for i,publico in enumerate(["Adultos", "Jovens"]):
    print(f"[{i+1}/{2}]")
    WOP.gera_mapa_frequencias(CEV="Ananindeua", publico=publico, exporta=True)

In [5]:
# Pega a lista de ovelhas já cadastradas de cada grupo
for i,grupo_id in enumerate(WOP.get_grupos_df()['id']):
    print(f"[{i+1}/{WOP.grupos_df["id"].size}] ")
    WOP.get_ovelhas(grupo_id,exporta=True)

[1/22] 
| Obtendo a lista de membros do grupo Fiat de Ananindeua
|-- Exportado para./grupos/Ananindeua_964_Fiat.csv
[2/22] 
| Obtendo a lista de membros do grupo Kadosh de Ananindeua
|-- Exportado para./grupos/Ananindeua_326_Kadosh.csv
[3/22] 
| Obtendo a lista de membros do grupo Pantocrator de Ananindeua
|-- Exportado para./grupos/Ananindeua_725_Pantocrator.csv
[4/22] 
| Obtendo a lista de membros do grupo São Tomé de Ananindeua
|-- Exportado para./grupos/Ananindeua_325_São Tomé.csv
[5/22] 
| Obtendo a lista de membros do grupo Israel de Ananindeua
|-- Exportado para./grupos/Ananindeua_324_Israel.csv
[6/22] 
| Obtendo a lista de membros do grupo Mãe da Misericórdia de Ananindeua
|-- Exportado para./grupos/Ananindeua_327_Mãe da Misericórdia.csv
[7/22] 
| Obtendo a lista de membros do grupo Santo Agostinho de Ananindeua
|-- Exportado para./grupos/Ananindeua_727_Santo Agostinho.csv
[8/22] 
| Obtendo a lista de membros do grupo Bom Pastor de São Brás
|-- Exportado para./grupos/São Brás_1

In [ ]:
path_driver = "msedgedriver.exe"

if path_driver: # Se for passado o path do driver
    driver_path = path_driver # usa esse path
    if "geckodriver" in driver_path.lower():
        sistema = "Linux"
    elif "msedgedriver" in driver_path.lower():
        sistema = "Windows" 

sistema